In [25]:
! pip install snowflake-connector-python
! pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import snowflake.connector
import os
import pandas as pd
from dotenv import load_dotenv


load_dotenv()

# Define connection parameters correctly
connection_param = {
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "account": "tm52713.ap-south-1",
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE")
}

# for testinbg 
print(f"Snowflake Warehouse: {connection_param['warehouse']}")
print(f"Snowflake Account: {connection_param['account']}")

session = snowflake.connector.connect(**connection_param)
cursor = session.cursor()
print("Connection successful")


Snowflake Warehouse: COMPUTE_WH
Snowflake Account: tm52713.ap-south-1
Connection successful


# Read the CSV file


In [2]:

csv_file_path = os.path.join("sample_data", "customers.csv") 
df = pd.read_csv(csv_file_path)

# function to check the data type to set datatype dynamicaly

In [3]:
def get_column_type(col, df):
    if df[col].dtype == 'int64':
        return 'INTEGER'
    elif df[col].dtype == 'float64':
        return 'FLOAT'
    else:
        return 'STRING'
    
# type dynamically
columns = df.columns.tolist()
column_definitions = ", ".join([f'"{col}" {get_column_type(col, df)}' for col in columns])
df.head()


,Customer ID,Customer Name,Region,Email,Phone,Industry,Customer City,Country,Zip
0,1,Lauren Barr,North,morganamy@gmail.com,001-508-070-2916x67243,Healthcare,Johnstonville,UK,77597
1,2,Mary Beard,South,shannon87@hotmail.com,420-109-5170,Education,Port Shelleyton,Australia,86104
2,3,Keith Rodriguez,North,charlesbarnes@reeves.com,001-789-191-8379x677,Hospitality,West Tracyfurt,India,57612
3,4,Karen Frazier,East,fhernandez@yahoo.com,+1-021-627-2409x9163,Education,Samuelport,France,8929
4,5,James Harrell,South,john44@mcbride.biz,091.780.9802x12362,Retail,Jessicabury,USA,30154


In [5]:
schema_name = connection_param["schema"] 
table_name = "customers"
cursor.execute(f"""
    SELECT COUNT(*) 
    FROM {schema_name}.INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_NAME = '{table_name.upper()}'
""")
table_exists = cursor.fetchone()[0] > 0
if table_exists:
    print(f"Table '{table_name}' already exists in db '{schema_name}'.")
else:
    create_table_query = f'CREATE TABLE {table_name} ({column_definitions})'
    cursor.execute(create_table_query)
    print(f"Table '{table_name}' created successfully.")

Table 'customers' already exists in db 'API_DB'.


Loading data intop snowflake 

In [18]:
# Construct INSERT query with placeholders
columns_str = ", ".join([f'"{col}"' for col in columns])
placeholders = ", ".join(["%s"] * len(columns))
insert_query = f'INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})'

# Convert DataFrame to list of tuples
data = [tuple(row) for row in df.itertuples(index=False, name=None)]

# Execute batch insert safely
cursor.executemany(insert_query, data)
print(f"Data inserted into '{table_name}' successfully.")

# Commit and close connection
session.commit()
cursor.close()
session.close()


Data inserted into 'customers' successfully.


# similarly for other csv file 


In [ ]:
import snowflake.connector
import os
import pandas as pd
from dotenv import load_dotenv


load_dotenv()

# Define connection parameters correctly
connection_param = {
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "account": "tm52713.ap-south-1",
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE")
}

# for testinbg 
print(f"Snowflake Warehouse: {connection_param['warehouse']}")
print(f"Snowflake Account: {connection_param['account']}")

session = snowflake.connector.connect(**connection_param)
cursor = session.cursor()
print("Connection successful")

csv_file_path = os.path.join("sample_data", "exchange_rates.csv") 
df = pd.read_csv(csv_file_path)
def get_column_type(col, df):
    if df[col].dtype == 'int64':
        return 'INTEGER'
    elif df[col].dtype == 'float64':
        return 'FLOAT'
    else:
        return 'STRING'
    
# type dynamically
columns = df.columns.tolist()
column_definitions = ", ".join([f'"{col}" {get_column_type(col, df)}' for col in columns])
# df.head()
# Your error "Database 'API_SCHEMA' does not exist or not authorized." means that Snowflake is treating API_SCHEMA as a database instead of a schema.
# This happens because in Snowflake, you must always specify the database first when referencing schemas and tables.
database_name = connection_param["database"] 
schema_name = connection_param["schema"] 
table_name = "exchange_rates"
cursor.execute(f"""
    SELECT COUNT(*) 
    FROM {database_name}.INFORMATION_SCHEMA.SCHEMATA 
    WHERE SCHEMA_NAME = '{schema_name.upper()}'
""")
schema_exists = cursor.fetchone()[0] > 0  

if schema_exists:
    print(f"Schema '{schema_name}' exists.")
    cursor.execute(f"USE SCHEMA {schema_name}")  
else:
    raise ValueError(f"Schema '{schema_name}' does not exist in Snowflake. Please check your connection settings.")

# cursor.execute(f"USE SCHEMA {schema_name}")

cursor.execute(f"""
    SELECT COUNT(*) 
    FROM {database_name}.INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_SCHEMA = '{schema_name.upper()}'
    AND TABLE_NAME = '{table_name.upper()}'
""")
table_exists = cursor.fetchone()[0] > 0
if table_exists:
    print(f"Table '{table_name}' already exists in db '{schema_name}'.")
else:
    create_table_query = f'CREATE TABLE {table_name} ({column_definitions})'
    cursor.execute(create_table_query)
    print(f"Table '{table_name}' created successfully.")
    # Construct INSERT query with placeholders
columns_str = ", ".join([f'"{col}"' for col in columns])
placeholders = ", ".join(["%s"] * len(columns))
insert_query = f'INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})'

# Convert DataFrame to list of tuples
data = [tuple(row) for row in df.itertuples(index=False, name=None)]

# Execute batch insert safely
cursor.executemany(insert_query, data)
print(f"Data inserted into '{table_name}' successfully.")

# Commit and close connection
session.commit()
cursor.close()
session.close()



Snowflake Warehouse: COMPUTE_WH
Snowflake Account: tm52713.ap-south-1
Connection successful
Schema 'API_SCHEMA' exists.
Table 'exchange_rates' already exists in db 'API_SCHEMA'.
Data inserted into 'exchange_rates' successfully.


# for fical_calender.csv

In [ ]:
import snowflake.connector
import os
import pandas as pd
from dotenv import load_dotenv


load_dotenv()

# Define connection parameters correctly
connection_param = {
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "account": "tm52713.ap-south-1",
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE")
}

# for testinbg 
print(f"Snowflake Warehouse: {connection_param['warehouse']}")
print(f"Snowflake Account: {connection_param['account']}")

session = snowflake.connector.connect(**connection_param)
cursor = session.cursor()
print("Connection successful")

csv_file_path = os.path.join("sample_data", "fiscal_calendar.csv") 
df = pd.read_csv(csv_file_path)
def get_column_type(col, df):
    if df[col].dtype == 'int64':
        return 'INTEGER'
    elif df[col].dtype == 'float64':
        return 'FLOAT'
    else:
        return 'STRING'
    
# type dynamically
columns = df.columns.tolist()
column_definitions = ", ".join([f'"{col}" {get_column_type(col, df)}' for col in columns])
# df.head()
# Your error "Database 'API_SCHEMA' does not exist or not authorized." means that Snowflake is treating API_SCHEMA as a database instead of a schema.
# This happens because in Snowflake, you must always specify the database first when referencing schemas and tables.
database_name = connection_param["database"] 
schema_name = connection_param["schema"] 
table_name = "fiscal_calendar"
cursor.execute(f"""
    SELECT COUNT(*) 
    FROM {database_name}.INFORMATION_SCHEMA.SCHEMATA 
    WHERE SCHEMA_NAME = '{schema_name.upper()}'
""")
schema_exists = cursor.fetchone()[0] > 0  

if schema_exists:
    print(f"Schema '{schema_name}' exists.")
    cursor.execute(f"USE SCHEMA {schema_name}")  # Now it's safe to use
else:
    raise ValueError(f"Schema '{schema_name}' does not exist in Snowflake. Please check your connection settings.")

# cursor.execute(f"USE SCHEMA {schema_name}")

cursor.execute(f"""
    SELECT COUNT(*) 
    FROM {database_name}.INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_SCHEMA = '{schema_name.upper()}'
    AND TABLE_NAME = '{table_name.upper()}'
""")
table_exists = cursor.fetchone()[0] > 0
if table_exists:
    print(f"Table '{table_name}' already exists in db '{schema_name}'.")
else:
    create_table_query = f'CREATE TABLE {table_name} ({column_definitions})'
    cursor.execute(create_table_query)
    print(f"Table '{table_name}' created successfully.")
    # Construct INSERT query with placeholders
columns_str = ", ".join([f'"{col}"' for col in columns])
placeholders = ", ".join(["%s"] * len(columns))
insert_query = f'INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})'

# Convert DataFrame to list of tuples
data = [tuple(row) for row in df.itertuples(index=False, name=None)]

# Execute batch insert safely
cursor.executemany(insert_query, data)
print(f"Data inserted into '{table_name}' successfully.")

# Commit and close connection
session.commit()
cursor.close()
session.close()



Snowflake Warehouse: COMPUTE_WH
Snowflake Account: tm52713.ap-south-1
Connection successful
Schema 'API_SCHEMA' exists.
Table 'fiscal_calendar' created successfully.
Data inserted into 'fiscal_calendar' successfully.


# orders.csv


In [ ]:
import snowflake.connector
import os
import pandas as pd
from dotenv import load_dotenv


load_dotenv()

# Define connection parameters correctly
connection_param = {
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "account": "tm52713.ap-south-1",
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE")
}

# for testinbg 
print(f"Snowflake Warehouse: {connection_param['warehouse']}")
print(f"Snowflake Account: {connection_param['account']}")

session = snowflake.connector.connect(**connection_param)
cursor = session.cursor()
print("Connection successful")

csv_file_path = os.path.join("sample_data", "orders.csv") 
df = pd.read_csv(csv_file_path)
def get_column_type(col, df):
    if df[col].dtype == 'int64':
        return 'INTEGER'
    elif df[col].dtype == 'float64':
        return 'FLOAT'
    else:
        return 'STRING'
    
# type dynamically
columns = df.columns.tolist()
column_definitions = ", ".join([f'"{col}" {get_column_type(col, df)}' for col in columns])
# df.head()
# Your error "Database 'API_SCHEMA' does not exist or not authorized." means that Snowflake is treating API_SCHEMA as a database instead of a schema.
# This happens because in Snowflake, you must always specify the database first when referencing schemas and tables.
database_name = connection_param["database"] 
schema_name = connection_param["schema"] 
table_name = "orders"
cursor.execute(f"""
    SELECT COUNT(*) 
    FROM {database_name}.INFORMATION_SCHEMA.SCHEMATA 
    WHERE SCHEMA_NAME = '{schema_name.upper()}'
""")
schema_exists = cursor.fetchone()[0] > 0 

if schema_exists:
    print(f"Schema '{schema_name}' exists.")
    cursor.execute(f"USE SCHEMA {schema_name}") 
else:
    raise ValueError(f"Schema '{schema_name}' does not exist in Snowflake. Please check your connection settings.")

# cursor.execute(f"USE SCHEMA {schema_name}")

cursor.execute(f"""
    SELECT COUNT(*) 
    FROM {database_name}.INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_SCHEMA = '{schema_name.upper()}'
    AND TABLE_NAME = '{table_name.upper()}'
""")
table_exists = cursor.fetchone()[0] > 0
if table_exists:
    print(f"Table '{table_name}' already exists in db '{schema_name}'.")
else:
    create_table_query = f'CREATE TABLE {table_name} ({column_definitions})'
    cursor.execute(create_table_query)
    print(f"Table '{table_name}' created successfully.")
    # Construct INSERT query with placeholders
columns_str = ", ".join([f'"{col}"' for col in columns])
placeholders = ", ".join(["%s"] * len(columns))
insert_query = f'INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})'

# Convert DataFrame to list of tuples
data = [tuple(row) for row in df.itertuples(index=False, name=None)]

# Execute batch insert safely
cursor.executemany(insert_query, data)
print(f"Data inserted into '{table_name}' successfully.")

# Commit and close connection
session.commit()
cursor.close()
session.close()



Snowflake Warehouse: COMPUTE_WH
Snowflake Account: tm52713.ap-south-1
Connection successful
Schema 'API_SCHEMA' exists.
Table 'orders' created successfully.
Data inserted into 'orders' successfully.


# products.csv

In [1]:
import snowflake.connector
import os
import pandas as pd
from dotenv import load_dotenv


load_dotenv()

# Define connection parameters correctly
connection_param = {
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "account": "tm52713.ap-south-1",
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE")
}

# for testinbg 
print(f"Snowflake Warehouse: {connection_param['warehouse']}")
print(f"Snowflake Account: {connection_param['account']}")

session = snowflake.connector.connect(**connection_param)
cursor = session.cursor()
print("Connection successful")

csv_file_path = os.path.join("sample_data", "products.csv") 
df = pd.read_csv(csv_file_path)
def get_column_type(col, df):
    if df[col].dtype == 'int64':
        return 'INTEGER'
    elif df[col].dtype == 'float64':
        return 'FLOAT'
    else:
        return 'STRING'
    
# type dynamically
columns = df.columns.tolist()
column_definitions = ", ".join([f'"{col}" {get_column_type(col, df)}' for col in columns])
# df.head()
# Your error "Database 'API_SCHEMA' does not exist or not authorized." means that Snowflake is treating API_SCHEMA as a database instead of a schema.
# This happens because in Snowflake, you must always specify the database first when referencing schemas and tables.
database_name = connection_param["database"] 
schema_name = connection_param["schema"] 
table_name = "products"
cursor.execute(f"""
    SELECT COUNT(*) 
    FROM {database_name}.INFORMATION_SCHEMA.SCHEMATA 
    WHERE SCHEMA_NAME = '{schema_name.upper()}'
""")
schema_exists = cursor.fetchone()[0] > 0 

if schema_exists:
    print(f"Schema '{schema_name}' exists.")
    cursor.execute(f"USE SCHEMA {schema_name}")  
else:
    raise ValueError(f"Schema '{schema_name}' does not exist in Snowflake. Please check your connection settings.")

# cursor.execute(f"USE SCHEMA {schema_name}")

cursor.execute(f"""
    SELECT COUNT(*) 
    FROM {database_name}.INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_SCHEMA = '{schema_name.upper()}'
    AND TABLE_NAME = '{table_name.upper()}'
""")
table_exists = cursor.fetchone()[0] > 0
if table_exists:
    print(f"Table '{table_name}' already exists in db '{schema_name}'.")
else:
    create_table_query = f'CREATE TABLE {table_name} ({column_definitions})'
    cursor.execute(create_table_query)
    print(f"Table '{table_name}' created successfully.")
    # Construct INSERT query with placeholders
columns_str = ", ".join([f'"{col}"' for col in columns])
placeholders = ", ".join(["%s"] * len(columns))
insert_query = f'INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})'

# Convert DataFrame to list of tuples
data = [tuple(row) for row in df.itertuples(index=False, name=None)]

# Execute batch insert safely
cursor.executemany(insert_query, data)
print(f"Data inserted into '{table_name}' successfully.")

# Commit and close connection
session.commit()
cursor.close()
session.close()



Snowflake Warehouse: COMPUTE_WH
Snowflake Account: tm52713.ap-south-1
Connection successful
Schema 'API_SCHEMA' exists.
Table 'products' created successfully.
Data inserted into 'products' successfully.
